In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns

from keras.models import load_model


from keras.layers import LSTM, Dense
from keras.models import Sequential
import keras.backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

from sklearn.model_selection import train_test_split

In [2]:
import socket 

IP = '127.0.0.1'
Sender_Receiver_comm_port = 9998

sender_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
sender_socket.connect((IP, Sender_Receiver_comm_port)) 

def start_sending_packets(send_Data):
    packet = "%d"%send_Data
    sender_socket.send( repr(packet).encode()) 
    data = sender_socket.recv(1024) 
    print('Packet sent to the receiver :',repr(data.decode("utf-8"))) 
    

# Train AI model with already extracted dataset

In [3]:
LR = glob.glob('C:/Users/hlee185/OneDrive - Georgia State University/2021 Fall/IoT/Dataset/LR_100/*.csv')
RL = glob.glob('C:/Users/hlee185/OneDrive - Georgia State University/2021 Fall/IoT/Dataset/RL_100/*.csv')
UP = glob.glob('C:/Users/hlee185/OneDrive - Georgia State University/2021 Fall/IoT/Dataset/UP_100/*.csv')
Down = glob.glob('C:/Users/hlee185/OneDrive - Georgia State University/2021 Fall/IoT/Dataset/Down_100/*.csv')

len(LR), len(RL), len(UP), len(Down)

(100, 100, 100, 100)

In [4]:
total_data = LR + RL + UP + Down
dataset = np.zeros((len(total_data), 100, 4))

for i, data in enumerate(total_data):
    dataset[i] = pd.read_csv(data, names=['n','a','b','c','d']).sample(n=100).sort_index().iloc[:,1:5]
    
label = np.array([[1, 0, 0, 0]] * len(LR) + [[0, 1, 0 ,0]] * len(RL) + [[0, 0, 1, 0]] * len(UP) + [[0, 0, 0, 1]] * len(Down))
dataset.shape, label.shape

((400, 100, 4), (400, 4))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, random_state=0, test_size=0.3)

callback = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.000001), 
    ModelCheckpoint(filepath='./weights.h5', save_best_only=True),
    EarlyStopping(monitor='val_acc', restore_best_weights=True, patience=5)
]

In [6]:
K.clear_session()

In [7]:
model_once = Sequential()
model_once.add(LSTM(20, input_shape=(100,4)))
model_once.add(Dense(64,activation='relu')) #tanh
model_once.add(Dense(16,activation='relu')) #tanh
model_once.add(Dense(4, activation='softmax'))
model_once.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.005), metrics=['acc'])

#result = model_once.fit(X_train, y_train, validation_split=0.33, epochs=30, batch_size=10, verbose=0)
result = model_once.fit(X_train, y_train, epochs=30, verbose=2)

Epoch 1/30
9/9 - 1s - loss: 1.4120 - acc: 0.2714
Epoch 2/30
9/9 - 0s - loss: 1.3440 - acc: 0.3571
Epoch 3/30
9/9 - 0s - loss: 1.1852 - acc: 0.5071
Epoch 4/30
9/9 - 0s - loss: 0.9313 - acc: 0.5036
Epoch 5/30
9/9 - 0s - loss: 0.9515 - acc: 0.5036
Epoch 6/30
9/9 - 0s - loss: 0.7720 - acc: 0.5929
Epoch 7/30
9/9 - 0s - loss: 0.7116 - acc: 0.6286
Epoch 8/30
9/9 - 0s - loss: 0.6470 - acc: 0.6786
Epoch 9/30
9/9 - 0s - loss: 0.5966 - acc: 0.7429
Epoch 10/30
9/9 - 0s - loss: 0.4980 - acc: 0.7607
Epoch 11/30
9/9 - 0s - loss: 0.3825 - acc: 0.8750
Epoch 12/30
9/9 - 0s - loss: 0.3647 - acc: 0.8536
Epoch 13/30
9/9 - 0s - loss: 0.2641 - acc: 0.9107
Epoch 14/30
9/9 - 0s - loss: 0.1405 - acc: 0.9607
Epoch 15/30
9/9 - 0s - loss: 0.0841 - acc: 0.9679
Epoch 16/30
9/9 - 0s - loss: 0.0516 - acc: 0.9893
Epoch 17/30
9/9 - 0s - loss: 0.0379 - acc: 0.9857
Epoch 18/30
9/9 - 0s - loss: 0.0257 - acc: 0.9964
Epoch 19/30
9/9 - 0s - loss: 0.0211 - acc: 0.9964
Epoch 20/30
9/9 - 0s - loss: 0.0150 - acc: 0.9964
Epoch 21/

In [8]:
model_once.evaluate(X_test, y_test)

predict_x=model_once.predict(X_test) 
classes_x_test=np.argmax(predict_x,axis=1)

4/4 [==============================] - 0s 4ms/step - loss: 0.1971 - acc: 0.9667


In [9]:
classes_y_test = []
for i in range(len(y_test)):
    for j in range(4):
        if y_test[i][j] == 1:
            classes_y_test.append(j)

count=0
print("Real value : Prediction\n---------------------------")
for i in range(len(X_test)):
    if(classes_x_test[i] != classes_y_test[i]):
        print("   ",classes_y_test[i], "\t   : \t", classes_x_test[i])
        count = count + 1
probab = count/len(X_test)
print("\n\nNumber of miscalculation : ",count,"/",len(classes_x_test),"=",round(probab,3))
print("\nAccuracy : ",((1-round(probab,4))*100),"%")
print("0 : LR, 1 : RL, 3 : UP, 4 : DOWN")

Real value : Prediction
---------------------------
    0 	   : 	 1
    1 	   : 	 2
    0 	   : 	 2
    0 	   : 	 1


Number of miscalculation :  4 / 120 = 0.033

Accuracy :  96.67 %
0 : LR, 1 : RL, 3 : UP, 4 : DOWN


# Real-time Dection & gesture classification 

In [12]:
import sys, serial, argparse, time
import os
import scipy
from scipy import signal
from collections import deque

### Deleting Serial Comm.

In [11]:
del ser

NameError: name 'ser' is not defined

In [13]:
ser = serial.Serial(
    port='com3',
    baudrate=19200,
)

In [14]:
def start_sensing_threshold(signal_lst, threshold, idx):
    outcome = False
    
    if signal_lst[idx - 1] > threshold:  #현재 데이터
        if signal_lst[idx - 2] < threshold:   #바로 전 데이터
            outcome = True
        else:
            pass

    return outcome
        
def end_sensing_threshold(signal_lst, threshold, idx):
    outcome = False
    
    if signal_lst[idx - 1] < threshold:  #현재 데이터
        if signal_lst[idx - 2] > threshold:   #바로 전 데이터
            outcome = True
        else:
            pass

    return outcome

def Classification_print(result):
    if result == 0:
        print("Classification Result is Left - Right")
    
    if result == 1:
        print("Classification Result is Right - Left")
    
    if result == 2:
        print("Classification Result is UP")
        
    if result == 3:
        print("Classification Result is Down")

In [15]:
sensor1, sensor2, sensor3, sensor4, index  = [], [], [], [], []
init_time = 6  #sec
window_size, Sampling_freq = 19, 53

init_period = Sampling_freq * init_time
Threshold = 16.5
hand_motion_period = 53

front_period, back_period = 50, 70

In [16]:
init_s1, init_s2, init_s3, init_s4, idx = [], [], [], [], 0
print("First Initialization")

for i in range(1,5):
    globals()['Test_threshold{}'.format(i)] = []

for i in range(100): #처음 100개 데이터는 거름
    tmp = ser.readline()

for i in range(int(init_period)):
    tmp = ser.readline()
    s1, s2, s3, s4 = tmp.decode()[:5], tmp.decode()[6:11], tmp.decode()[12:17], tmp.decode()[18:]

    try:
        s1, s2, s3, s4 = (int(s1)-10000), (int(s2)-10000), (int(s3)-10000), (int(s4)-10000)
    
    except:
        print("Error at index ", i)
        pass
    
    if type(s1) == int and type(s2) == int and type(s3) == int and type(s4) == int: 
        index.append(idx)
        idx = idx + 1
        
        for j in range(1,5):
            globals()['Test_threshold{}'.format(j)].append(0)
            globals()['init_s{}'.format(j)].append(globals()['s{}'.format(j)])
            globals()['sensor{}'.format(j)].append(globals()['s{}'.format(j)])

offset_s1, offset_s2, offset_s3, offset_s4 = np.mean(init_s1), np.mean(init_s2), np.mean(init_s3), np.mean(init_s4)
init_s1, init_s2, init_s3, init_s4 = [], [], [], []

First Initialization


## Real Time operation 

In [17]:
for i in range(1,5):
    globals()['sensor{}_frame'.format(i)] = []  
    globals()['S{}_Threshold'.format(i)], globals()['s{}_start_frame'.format(i)], globals()['s{}_end_frame'.format(i)] = Threshold, 0, 0
    globals()['s{}_cnt'.format(i)], globals()['s{}_safe_cnt'.format(i)], globals()['s{}_safe_cnt_2'.format(i)] = 0, 0, 0
    
Final_frame, safe_cnt3, start_frame, end_frame, next_frame_period, cnt = [], 0, 0, 0, 0, 0

while True:
    if ser.readable():
        res = ser.readline()
        s1, s2, s3, s4 = res.decode()[:5], res.decode()[6:11], res.decode()[12:17], res.decode()[18:]
                        
        try:
            s1, s2, s3, s4 = (int(s1)-10000), (int(s2)-10000), (int(s3)-10000), (int(s4)-10000)

        except:
            print(idx, " index has Error")
            pass
        
        if type(s1) == int and type(s2) == int and type(s3) == int and type(s4) == int: 
            index.append(idx)
            
            for i in range (1,5): 
                globals()['Test_threshold{}'.format(i)].append(globals()['S{}_Threshold'.format(i)])
                
                tmp2 = globals()['s{}'.format(i)] - globals()['offset_s{}'.format(i)]
                tmp2 = np.abs(np.round(tmp2,3))
                globals()['sensor{}'.format(i)].append(tmp2)
                
                if globals()['s{}_start_frame'.format(i)] == 0 and globals()['s{}_end_frame'.format(i)] == 0:  #안정해졌을 때 offset 계산해야 함
                    globals()['init_s{}'.format(i)].append(globals()['s{}'.format(i)])
                #-----------------------------------------------------------------------------------------------------------------------------------------
                
                if start_sensing_threshold(globals()['sensor{}'.format(i)], globals()['S{}_Threshold'.format(i)], idx) == True:  # Hand motion detection 시작
                    globals()['s{}_start_frame'.format(i)] = idx - 70 #period 설정, 약 20 = 0.4 초 정도 앞으로
                    globals()['s{}_safe_cnt_2'.format(i)] = globals()['s{}_safe_cnt_2'.format(i)] + 1
                    print(i, " 센서 넘음", idx)
                
                elif globals()['sensor{}'.format(i)][idx] > globals()['S{}_Threshold'.format(i)]: # Hand motion detection 중간
                    globals()['s{}_cnt'.format(i)], globals()['s{}_safe_cnt'.format(i)] = (globals()['s{}_cnt'.format(i)] + 1), (globals()['s{}_safe_cnt'.format(i)] + 1)
                    
                elif end_sensing_threshold(globals()['sensor{}'.format(i)], globals()['S{}_Threshold'.format(i)], idx) == True and globals()['s{}_cnt'.format(i)] > 0 and globals()['s{}_safe_cnt_2'.format(i)] > 0:  # Hand motion detection 끝
                    globals()['s{}_end_frame'.format(i)] = idx + 70  #0.6초 정도 뒤로
                    print(i, " 센서 넘었다가 다시 내려감", idx)
                    globals()['s{}_safe_cnt'.format(i)] = 0
                    
                elif globals()['s{}_start_frame'.format(i)] == 0 and globals()['s{}_end_frame'.format(i)] == 0: #False 일 때, end_Frame 끝나면 ㄱ
                    if idx % init_period == 0:  #매번 5초 마다 init 실행
                        globals()['offset_s{}'.format(i)] = np.mean(globals()['init_s{}'.format(i)])
                        idx_tmp = len(globals()['sensor{}'.format(i)])
                        globals()['S{}_Threshold'.format(i)] = np.mean(globals()['sensor{}'.format(i)][idx_tmp-50:idx_tmp]) + Threshold  #Th업데이트
                        globals()['init_s{}'.format(i)] = []  #offset 계산해주는 리스트 초기화
                        
                        print(i," 센서 ", idx, " 번째 idx 에서 Init, Threshold 업데이트!!! : ", np.round(globals()['S{}_Threshold'.format(i)], 3))
                
                if globals()['sensor{}'.format(i)][idx] > globals()['S{}_Threshold'.format(i)] and globals()['s{}_safe_cnt'.format(i)] > 50: # 안전장치: 계속 넘으면 다시 안내려감 = 1초 동안 넘으면 다시 내려감
                    print(i, " 센서 안전 장치! ")  #sensor 값에서 바로 추출
                    idx_tmp = len(globals()['sensor{}'.format(i)])
                    globals()['offset_s{}'.format(i)] = np.mean(globals()['sensor{}'.format(i)][idx_tmp-50:idx_tmp]) #offset 다시 설정
                    globals()['s{}_safe_cnt'.format(i)] = 0
                
                if idx == globals()['s{}_end_frame'.format(i)] and idx > 800:  #대략 맨 처음 8초 정도
                    #print(i," 센서 => 리스트에 저장")
                    globals()['sensor{}_frame'.format(i)].append([globals()['s{}_start_frame'.format(i)], globals()['s{}_end_frame'.format(i)]]) 
                    globals()['s{}_end_frame'.format(i)], globals()['s{}_start_frame'.format(i)], globals()['s{}_cnt'.format(i)], globals()['s{}_safe_cnt_2'.format(i)] = 0, 0, 0, 0  #초기화
            
#-----------------------------------------------------------------------------------------------------------------------------------------           
            #맨 처음 넘은 건 바로 저장, 마지막 시점은 계속 업데이트
            if next_frame_period < idx:
                
                for k in range(1,5):
                    if not globals()['sensor{}_frame'.format(k)]:    #빈리스트 체크 = True 일 때 pass
                        pass
                    
                    else:     #빈리스트가 아니면 
                        start_frame = globals()['sensor{}_frame'.format(k)][-1][0]   #맨처음만 저장    
                        end_frame = globals()['sensor{}_frame'.format(k)][-1][1]   #계속 업데이트                    
                        
                if  idx == end_frame and start_frame != 0 and end_frame != 0:  #start_frame + 100? end_frame?
                    Final_frame.append([start_frame, end_frame])
                    print("Final Frame ===============> ", Final_frame[-1])
                    for k in range(1,5):
                        globals()['sensor{}_frame'.format(k)] = []  #초기화
                        
                    start_frame, end_frame, next_frame_period = 0, 0, idx + 400
                    
#----------------------------------------------------------------------------------------------------------------------------------------------------                    
                    df_frame, cnt = [], cnt+1
                    for k in range (1,5):
                        df_frame.append(globals()['sensor{}'.format(k)][Final_frame[-1][0]:Final_frame[-1][1]])
                    df_frame = pd.DataFrame(df_frame)
                    df_frame = df_frame.transpose()
                    tmp_name = 'C:/Users/hlee185/OneDrive - Georgia State University/2021 Fall/IoT/Dataset/test5/Down/data_%d.csv'%cnt
                    df_frame.to_csv(tmp_name)
                    data = np.array(pd.read_csv(tmp_name, names=['n','a','b','c','d']).sample(n=100).sort_index().iloc[:,1:5])
                    data_classify = data.reshape(1,100,4)

                    classification = model_once.predict(data_classify) 
                    result = np.argmax(classification,axis=1)
                    
                    Classification_print(result)
                    
                    start_sending_packets(result)
                    
                    
                    
                    
                    
            idx = idx + 1
            

1  센서  318  번째 idx 에서 Init, Threshold 업데이트!!! :  26.745
2  센서  318  번째 idx 에서 Init, Threshold 업데이트!!! :  29.397
3  센서  318  번째 idx 에서 Init, Threshold 업데이트!!! :  29.666
4  센서  318  번째 idx 에서 Init, Threshold 업데이트!!! :  25.468
1  센서  636  번째 idx 에서 Init, Threshold 업데이트!!! :  18.84
2  센서  636  번째 idx 에서 Init, Threshold 업데이트!!! :  17.36
3  센서  636  번째 idx 에서 Init, Threshold 업데이트!!! :  18.62
4  센서  636  번째 idx 에서 Init, Threshold 업데이트!!! :  18.86
1  센서  954  번째 idx 에서 Init, Threshold 업데이트!!! :  19.207
2  센서  954  번째 idx 에서 Init, Threshold 업데이트!!! :  18.305
3  센서  954  번째 idx 에서 Init, Threshold 업데이트!!! :  17.097
4  센서  954  번째 idx 에서 Init, Threshold 업데이트!!! :  17.154
1  센서 넘음 1193
4  센서 넘음 1236
1  센서 안전 장치! 
1  센서 넘었다가 다시 내려감 1245
4  센서 넘었다가 다시 내려감 1263
2  센서  1272  번째 idx 에서 Init, Threshold 업데이트!!! :  23.078
3  센서  1272  번째 idx 에서 Init, Threshold 업데이트!!! :  27.695
Final Frame ===============>  [1123, 1315]
Classification Result is Down
Packet sent to the receiver : "'3'"
1  센서 넘음 1416
1  센서 넘

1  센서 넘었다가 다시 내려감 9864
1  센서 넘음 9871
1  센서 넘었다가 다시 내려감 9872
1  센서 넘음 9874
1  센서 넘음 9880
1  센서 넘었다가 다시 내려감 9882
1  센서 넘음 9884
1  센서 넘었다가 다시 내려감 9887
4  센서 안전 장치! 
4  센서 넘었다가 다시 내려감 9892
1  센서 넘음 9907
1  센서 넘었다가 다시 내려감 9910
1  센서 넘음 9912
1  센서 넘었다가 다시 내려감 9914
1  센서 넘음 9983
1  센서 넘었다가 다시 내려감 9984
1  센서 넘음 10011
1  센서 넘었다가 다시 내려감 10012
1  센서 넘음 10058
1  센서 넘었다가 다시 내려감 10059
1  센서 넘음 10080
1  센서 넘었다가 다시 내려감 10085
1  센서 넘음 10109
1  센서 넘었다가 다시 내려감 10110
1  센서 넘음 10128
1  센서 넘었다가 다시 내려감 10131
1  센서 넘음 10137
1  센서 넘었다가 다시 내려감 10139
1  센서 넘음 10151
1  센서 넘었다가 다시 내려감 10152
2  센서  10176  번째 idx 에서 Init, Threshold 업데이트!!! :  21.655
3  센서  10176  번째 idx 에서 Init, Threshold 업데이트!!! :  28.976
4  센서  10176  번째 idx 에서 Init, Threshold 업데이트!!! :  26.012
1  센서 넘음 10193
1  센서 넘었다가 다시 내려감 10194
1  센서 넘음 10201
1  센서 넘었다가 다시 내려감 10204
1  센서 넘음 10394
1  센서 넘었다가 다시 내려감 10396
1  센서 넘음 10416
1  센서 넘었다가 다시 내려감 10419
1  센서 넘음 10422
1  센서 넘었다가 다시 내려감 10423
1  센서 넘음 10443
1  센서 넘었다가 다시 내려감 10449
2  센서  10494  번째 idx 에서

KeyboardInterrupt: 